<h1 style="text-align:center;">ESCUELA POLITÉCNICA NACIONAL</h1>
<h2 style="text-align:center;">FACULTAD DE INGENIERÍA DE SISTEMAS</h2>
<h3 style="text-align:center;">INGENIERÍA EN CIENCIAS DE LA COMPUTACIÓN</h3>

**PERÍODO ACADÉMICO:** 2025-A  
**ASIGNATURA:** ICCD412 Métodos Numéricos  
**GRUPO:** GR2  
**TIPO DE INSTRUMENTO:** Proyecto   
**FECHA DE ENTREGA LÍMITE:** ..   
**ALUMNO:** Murillo Juan, Sangucho Eddy, Morales Presly

## Librerías a utilizar
* pandas 
* seaborn 
* matplotlib
* numpy
* scipy 

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats as statsCorr

# Limpieza DE CSV

## Creación del DataFrame
Utilizamos pandas para crear un DataFrame a partir del csv.

In [ ]:
df = pd.read_csv("Train.csv")

## Revisión del tipo de datos
Realizamos esta revisión porque pandas asigna un tipo de dato que él considera adecuado y a veces el tipo de dato es erróneo.

In [ ]:
print(df.dtypes)

## Conclusión del paso anterior
Nos damos cuenta de que están mal asignadas las categóricas y el date_time no está en formato datetime64.

In [ ]:
df["date_time"] = df["date_time"].astype("datetime64[ns]")
df["weather_type"] = df["weather_type"].astype("category")
df["weather_description"] = df["weather_description"].astype("category")

## Limpieza de columnas duplicadas
Revisamos el número de datos de cada categoría de weather_description y nos damos cuenta de que hay un duplicado sky is clear y Sky is clear

In [ ]:
print(df['weather_description'].value_counts())

In [ ]:
df["weather_description"] = (df["weather_description"].str.lower()).str.strip()
df["weather_description"] = df["weather_description"].astype("category")
print(df['weather_description'].value_counts())

## Trabajo de limpieza con la variable temperatura

In [ ]:
#Eliminamos la columna de isHoliday porque aporta ruido y porque no lo consideramos para el modelo, ademas vimos que existen solo 43 datos de esta sobre 33750
df.drop(columns="is_holiday", inplace=True)#Ponemos inplace=True porque sino no cambia el df

#Aca veremos las variables categoricas y su relacion ()
df_test = df[["weather_type","weather_description", "traffic_volume"]]
# Solo mantener combinaciones que realmente aparecen en los datos
df_grp = df_test.groupby(["weather_type", "weather_description"], as_index=False, observed=True).mean()
print(df_grp)

#Vemos el tamanio para ver que metodo de categorico a numerico podemos usar 
#df_count = df_test.groupby(["weather_type", "weather_description"], as_index=False, observed=True).size()
#print(df_count)


#std Desviacion estandar (Estadistica)
#print(df.describe(include="all")) #-> De aqui influimos que temperature es en kelvin por los min y max   #Trabajamos con Kelvin para el calculo, transformamos a celsius para las presentaciones


#Dado que no se cuenta con información sobre la ubicación geográfica o el clima específico del lugar donde se recolectaron los datos 
#Por tanto, para evitar sesgar el análisis y el modelo, se opta por conservar estos valores. 
#Solo se marcan como NaN, como 0 K, que corresponden claramente a errores de medición o registros faltantes.

df.loc[df["temperature"] == 0, "temperature"] = pd.NA 
#print(df.isnull().sum())

#Como son 10 los rellenamos con la media de la hora. Métodos avanzados pueden introducir sesgos o sobreajustar en conjuntos pequeños o con poca información.
df["temperature"] = df.groupby(df["date_time"].dt.hour)["temperature"].transform(lambda x: x.fillna(x.mean()))


#Info como memoria etc
df.info()
print(df.isnull().sum())


#Diagrama de caja
sns.boxplot(x="weather_type", data=df)

## Dividimos los datos en 80/20 para hacerTrain y Test 

Esto lo realizamos para comparar los resultados de nuestro modelo con los medidos y registrados en la tabla y ver cuanta exactitud tiene.


In [ ]:
df = df.sample(frac=1, random_state=42)#Barajeamos las filas
cut = int(len(df) * 0.8)
df_train = df.iloc[:cut].copy() #80%
df_test  = df.iloc[cut:].copy() #20%

## Unir weather_type y  weather_description
Unimos ambos porque están directamente relacionados y porque para simplificar la dimensión de la tabla

In [ ]:
# Solo mantener combinaciones que realmente aparecen en los datos y tienen traffic volume
df_test_sub = df[["weather_type","weather_description", "traffic_volume"]]
df_grp = df_test_sub.groupby(["weather_type", "weather_description"], as_index=False, observed=True).mean()
df_count = df_test_sub.groupby(["weather_type", "weather_description"], as_index=False, observed=True).size()
print(df_count)

## Conclusión del paso anterior
Vemos que no podemos realizar One-Hot como método para pasar de variable categórica a numérica porque tenemos muchos valores en ciertas filas y en otros tenemos muy pocos.
Además, recurrimos al método Target Encoding modificado porque el normal puede producir goteo.

## Target Encoding 

Explication

#Codigo

In [ ]:
#Por libreria

## Matriz de correlación

Explicación y decir cuáles elegimos con base en 

In [ ]:
#Codigo 

## Matriz de correlación de Weather type y Weather Description
Explicar que la nieve tiene más

In [ ]:
#Codigo

In [ ]:
## Volumen de Trafico segun el Tipo de Clima Explicar:

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x="weather_type", y="traffic_volume", data=df)
plt.xticks(rotation=45)
plt.title("Volumen de Tráfico según Tipo de Clima")
plt.tight_layout()
plt.show()

## Relasion Humedad vs Volumen de trafico 

In [ ]:
sns.regplot(
    x="humidity",
    y="traffic_volume",
    data=df,
    line_kws={'color': 'red'}
)
plt.ylim(0,)

plt.show()